# Parlament Speaker Recogniction Model
This notebook should predict what speaker said certain things based on the Austrian parlament speaches. 

This small project is based on the Dataset assembled by Mario Zechner (https://marioslab.io/)

In [1]:
# DATA FROM SPEAKER
import pandas as pd

# Either use direct download or load locally https://woswormeileistung.marioslab.io/data/persons.json
#speaker = pd.read_json("https://woswormeileistung.marioslab.io/data/persons.json")
speaker = pd.read_json("persons.json")
print(speaker.shape)
speaker.head()

(681, 6)


,id,name,parties,periods,url,imageUrl
0,3612,Sonja Ablinger,[SPÖ],"[XX, XXIII, XXIV]",https://parlament.gv.at/person/3612,https://parlament.gv.at/dokument/bild/34898/34...
1,14854,Dipl.-Ing. Elke Achleitner,"[BZÖ, FPÖ]",[XXII],https://parlament.gv.at/person/14854,https://parlament.gv.at/dokument/bild/21013/21...
2,83119,Mag. Nikolaus Alm,"[LIF, NEOS]",[XXV],https://parlament.gv.at/person/83119,https://parlament.gv.at/dokument/bild/43808/43...
3,51879,"Mag. Hannes Amesbauer, BA",[FPÖ],"[XXVI, XXVII]",https://parlament.gv.at/person/51879,https://parlament.gv.at/dokument/bild/200696/2...
4,2819,"Werner Amon, MBA",[ÖVP],"[XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XXVI]",https://parlament.gv.at/person/2819,https://parlament.gv.at/dokument/bild/90356/90...


As one can see there are about 681 Speakers from the austrian parlament, starting from 2002

In [2]:
# DATA FROM SESSIONS 
# BE AWARA: sessions data has 440 mb, so it is only imported locally 
# https://woswormeileistung.marioslab.io/data/sessions.json

sessions = pd.read_json("sessions.json")

print(sessions.shape)
sessions.head()

(1001, 7)


,url,period,sessionNumber,sessionLabel,date,protocolUrls,sections
0,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,251,251. Sitzung (251/NRSITZ),2024-01-31,[],[]
1,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,250,250. Sitzung (250/NRSITZ),2024-01-31,[],[]
2,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,249,249. Sitzung (249/NRSITZ),2024-01-31,[],[]
3,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,248,248. Sitzung (248/NRSITZ),2023-12-15,[],[]
4,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,247,247. Sitzung (247/NRSITZ),2023-12-15,[],[]


In [3]:
sessions.sections.head()

0    []
1    []
2    []
3    []
4    []
Name: sections, dtype: object

In [4]:
sessions.sections.drop_duplicates()

0                                                      []
35      [{'speaker': '88386', 'text': 'Ich darf die 21...
36      [{'speaker': '88386', 'text': 'Meine sehr geeh...
39      [{'speaker': '88386', 'text': 'Ich eröffne die...
40      [{'speaker': '88386', 'text': 'Meine sehr geeh...
                              ...                        
996     [{'speaker': '799', 'text': 'Ich eröffne die 5...
997     [{'speaker': '799', 'text': 'Die Sitzung ist e...
998     [{'speaker': '799', 'text': 'Meine Damen und H...
999     [{'speaker': '799', 'text': 'Die Sitzung ist e...
1000    [{'speaker': '334', 'text': 'Meine Damen und H...
Name: sections, Length: 955, dtype: object

In [5]:
sessions.sections.describe()

count     1001
unique     955
top         []
freq        47
Name: sections, dtype: object

In [6]:
session_na = [1 if sessions["sections"][i] == [] else 0 for i in range(len(sessions))]
print("Absolute number of na: "+ str(sum(session_na)) + " || relative number of na: "+ str(sum(session_na)/len(sessions)))

Absolute number of na: 47 || relative number of na: 0.04695304695304695


In [7]:
# the entries in "speaker" arer list of dicts; therefore loop over the list to extract the dicts
text_dict = [j  for sections in sessions["sections"] for j in sections] 

# make a pandas df from the extracted dict
text = pd.DataFrame.from_dict(text_dict, orient="columns")
text["input"] = "SPEAKER: "+text["speaker"]+"; TEXT: "+text["text"]
print(text.shape) # 173498 
text.head()

(173498, 5)


,speaker,text,callouts,links,input
0,88386,Ich darf die 216. Sitzung des Nationalrates um...,"[{'caller': '2343', 'text': 'Ich bin noch nich...",[],SPEAKER: 88386; TEXT: Ich darf die 216. Sitzun...
1,88386,Hinsichtlich der eingelangten Verhandlungsgege...,[],"[{'label': '14141/AB', 'url': 'https://parlame...",SPEAKER: 88386; TEXT: Hinsichtlich der eingela...
2,88386,"Außerdem weise ich\n\ndie Anträge 3425/A, 3426...",[],"[{'label': '3425/A', 'url': 'https://parlament...",SPEAKER: 88386; TEXT: Außerdem weise ich\n\ndi...
3,88386,"Weiters darf ich mitteilen, dass folgende Fris...",[],"[{'label': '1774 der Beilagen', 'url': 'https:...",SPEAKER: 88386; TEXT: Weiters darf ich mitteil...
4,88386,Wir kommen nunmehr zur Abstimmung über den Ant...,[],"[{'label': '1774 der Beilagen', 'url': 'https:...",SPEAKER: 88386; TEXT: Wir kommen nunmehr zur A...


In [8]:
#for i in range(10): 
#    print(text["callouts"][i] == [])
tryout = pd.DataFrame([text["callouts"][i] == [] for i in range(10)])
tryout.describe()

,0
count,10
unique,2
top,True
freq,9


# Try using Hugging Face transformers library

In [9]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(text[:100])
ds

Dataset({
    features: ['speaker', 'text', 'callouts', 'links', 'input'],
    num_rows: 173498
})

In [10]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
model_nm = 'microsoft/deberta-v3-small'
t#okz = AutoTokenizer.from_pretrained(model_nm)

def tok_func(x): return tokz(x["input"])
#tok_ds = ds.map(tok_func, batched=True)
#tok_ds

/home/georg/miniforge3/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
# MY JOURNEY WITH THE HUGGING FACE LIBRARY STOPS HERE BECAUSE OF AN ERROR

#from transformers import TrainingArguments,Trainer

#args = TrainingArguments('outputs', 
#                         learning_rate=8e-5, 
#                         warmup_ratio=0.1, 
#                         lr_scheduler_type='cosine', 
#                         fp16=False,
#                         evaluation_strategy="epoch", 
#                         per_device_train_batch_size=128, 
#                         per_device_eval_batch_size=128*2,
#                         num_train_epochs=4, 
#                         weight_decay=0.01, 
#                         report_to='none')

In [12]:
#model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
#trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],tokenizer=tokz, compute_metrics=corr_d)

# Try the speaker classification with fast.ai library

In [13]:
from fastai.text.core import tokenize_df
train_df = text[:100]
text_tok = tokenize_df(df=train_df,text_cols="text")

In [14]:
text_tok[0]

,speaker,callouts,links,input,text,text_length
0,88386,"[{'caller': '2343', 'text': 'Ich bin noch nicht fertig, Herr Präsident!'}, {'text': 'Beifall bei SPÖ und Grünen.'}, {'text': 'In Richtung Galerie:'}]",[],"SPEAKER: 88386; TEXT: Ich darf die 216. Sitzung des Nationalrates um 16.45 Uhr eröffnen. (Abg. Hoyos-Trauttmansdorff: Ich bin noch nicht fertig, Herr Präsident!)\n\nIch darf alle Damen und Herren Abgeordnete und auch unsere Gäste auf der Galerie und zu Hause vor den Fernsehschirmen recht herzlich begrüßen. Die Journalisten haben sich schon – bis auf einen, glaube ich – verabschiedet. Ja, einer hält wacker die Stellung. (Beifall bei SPÖ und Grünen.) – (In Richtung Galerie:) Der Applaus ist Ihrer!\n\nDie Sitzung ist eröffnet.\n\nAls verhindert gemeldet sind die Abgeordneten Mag. Wolfgang Ger...","[xxbos, xxmaj, ich, darf, die, 216, ., , xxmaj, sitzung, des, xxmaj, nationalrates, um, 16.45, , xxmaj, uhr, eröffnen, ., (, abg, ., hoyos, -, trauttmansdorff, :, xxmaj, ich, bin, noch, nicht, fertig, ,, xxmaj, herr, xxmaj, präsident, !, ), \n\n, xxmaj, ich, darf, alle, xxmaj, damen, und, xxmaj, herren, xxmaj, abgeordnete, und, auch, unsere, xxmaj, gäste, auf, der, xxmaj, galerie, und, zu, xxmaj, hause, vor, den, xxmaj, fernsehschirmen, recht, herzlich, begrüßen, ., xxmaj, die, xxmaj, journalisten, haben, sich, schon, , –, bis, auf, einen, ,, glaube, ich, , –, verabschiedet, ., xxmaj, ...",236
1,88386,[],"[{'label': '14141/AB', 'url': 'https://parlament.gv.at/pls/portal/hi.link?pfad=/PAKT/VHG/XXVII/AB/AB_14141/index.shtml'}, {'label': '14148/AB', 'url': 'https://parlament.gv.at/pls/portal/hi.link?pfad=/PAKT/VHG/XXVII/AB/AB_14148/index.shtml'}, {'label': '3415/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03415/index.shtml'}, {'label': '3410/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03410/index.shtml'}, {'label': '3422/A(E)', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03422/index.shtml'}, {'label': '3408/A(E)', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_...","SPEAKER: 88386; TEXT: Hinsichtlich der eingelangten Verhandlungsgegenstände und deren Zuweisungen verweise ich gemäß § 23 Abs. 4 der Geschäftsordnung auf die im Sitzungssaal verteilte Mitteilung.\n\nDie schriftliche Mitteilung hat folgenden Wortlaut:\n\nA. Eingelangte Verhandlungsgegenstände:\n\nAnfragebeantwortungen: 14141/AB bis 14148/AB\n\nB. Zuweisungen in dieser Sitzung:\n\nzur Vorberatung:\n\nAusschuss für Arbeit und Soziales:\n\nAntrag der Abgeordneten Tanja Graf, Mag. Markus Koza, Kolleginnen und Kollegen betreffend ein Bundesgesetz, mit dem das Ausländerbeschäftigungsgesetz geände...","[xxbos, xxmaj, hinsichtlich, der, eingelangten, xxmaj, verhandlungsgegenstände, und, deren, xxmaj, zuweisungen, verweise, ich, gemäß, §, , 23, xxmaj, abs, ., , 4, der, xxmaj, geschäftsordnung, auf, die, i, m, xxmaj, sitzungssaal, verteilte, xxmaj, mitteilung, ., \n\n, xxmaj, die, schriftliche, xxmaj, mitteilung, hat, folgenden, xxmaj, wortlaut, :, \n\n, xxup, a., xxmaj, eingelangte, xxmaj, verhandlungsgegenstände, :, \n\n, xxmaj, anfragebeantwortungen, :, 14141, /, xxup, ab, bis, 14148, /, xxup, ab, \n\n, xxup, b., xxmaj, zuweisungen, in, dieser, xxmaj, sitzung, :, \n\n, zur, xxmaj, vorb...",913
2,88386,[],"[{'label': '3425/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03425/index.shtml'}, {'label': '3426/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03426/index.shtml'}, {'label': '3427/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03427/index.shtml'}, {'label': '3428/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03428/index.shtml'}, {'label': '3429/A', 'url': 'https://parlament.gv.at/PAKT/VHG/XXVII/A/A_03429/index.shtml'}]","SPEAKER: 88386; TEXT: Außerdem weise ich\n\ndie Anträge 3425/A, 3426/A und 3427/A dem Ausschuss für Wirtschaft, Industrie und Energie,\n\nden Antrag 3428/A dem Umweltausschuss sowie\n\nden Antrag 3429/A dem Budgetausschuss\n\nzu.\n\n*****\n\nFristsetzungsanträge","[xxbos, xxmaj, außerdem, we

In [15]:
from fastai.text.data import TextDataLoaders
dl = TextDataLoaders.from_df(train_df, valid_pct=0.2, seed=None,text_col="text", label_col="speaker")
dl.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj hinsichtlich der xxunk xxmaj xxunk und xxunk xxmaj zuweisungen xxunk ich xxunk xxunk xxunk xxmaj xxunk . xxunk der xxmaj xxunk auf die i m xxmaj xxunk xxunk xxmaj mitteilung . \n\n xxmaj die xxunk xxmaj mitteilung hat xxunk xxmaj xxunk : \n\n xxup xxunk xxmaj xxunk xxmaj xxunk : \n\n xxmaj anfragebeantwortungen : xxunk / xxup ab bis xxunk / xxup ab \n\n xxup xxunk xxmaj zuweisungen in dieser xxmaj sitzung : \n\n zur xxmaj xxunk : \n\n xxmaj ausschuss für xxmaj arbeit und xxmaj xxunk : \n\n xxmaj antrag der xxmaj abgeordneten xxmaj tanja xxmaj graf , xxmaj mag . xxmaj markus xxmaj koza , xxmaj kolleginnen und xxmaj kollegen betreffend ein xxmaj bundesgesetz , mit dem das xxmaj xxunk geändert wird ( xxunk / xxup a ) \n\n xxmaj xxunk : \n\n xxmaj antrag der xxmaj abgeordneten xxmaj mag .",88386
1,"xxbos xxmaj wir kommen xxunk zur xxmaj abstimmung über den xxmaj antrag der xxmaj abgeordneten xxmaj lukas xxmaj hammer und xxmaj tanja xxmaj graf , dem xxmaj ausschuss für xxmaj wirtschaft , xxmaj industrie und xxmaj energie zur xxmaj berichterstattung über die xxmaj xxunk xxunk der xxmaj xxunk eine xxmaj frist bis zum 31 . xxmaj mai 2023 zu setzen . \n\n xxmaj wer dafür ist , den bitte ich um ein xxunk xxmaj zeichen der xxmaj xxunk . – xxmaj das ist angenommen . \n\n xxmaj wir kommen zur xxmaj abstimmung über den xxmaj antrag der xxmaj abgeordneten xxmaj lukas xxmaj hammer und xxmaj tanja xxmaj graf , dem xxmaj ausschuss für xxmaj wirtschaft , xxmaj industrie und xxmaj energie zur xxmaj berichterstattung über den xxmaj antrag 3426 / a eine xxmaj frist bis zum 31 . xxmaj mai 2023",88386


In [39]:
from fastai.metrics import accuracy,Perplexity
from fastai.text.learner import text_classifier_learner
from fastai.text.models import AWD_LSTM
learn = text_classifier_learner(dl, AWD_LSTM, metrics=[accuracy, Perplexity()])

In [41]:
from fastai.text.all import *
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


IOStream.flush timed out


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/georg/miniforge3/lib/python3.10/site-packages/fastai/data/transforms.py", line 261, in encodes
    return TensorCategory(self.vocab.o2i[o])
KeyError: '83121'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/georg/miniforge3/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/georg/miniforge3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 41, in fetch
    data = next(self.dataset_iter)
  File "/home/georg/miniforge3/lib/python3.10/site-packages/fastai/data/load.py", line 138, in create_batches
    yield from map(self.do_batch, self.chunkify(res))
  File "/home/georg/miniforge3/lib/python3.10/site-packages/fastcore/basics.py", line 230, in chunked
    res = list(itertools.islice(it, chunk_sz))
  Fil

In [ ]:
learn.predict("Ich bin Wolfgang Sobotka.")

In [44]:
#speaker["id"] == 18140
speaker.loc[speaker["id"]==18140]

,id,name,parties,periods,url,imageUrl
600,18140,Mag. Dr. Martin Kocher,[],[XXVII],https://parlament.gv.at/person/18140,https://parlament.gv.at/dokument/bild/200504/20050490_384.jpg


In [47]:
len(train_df["speaker"].unique())

17